# Textual Inversion with Determined

This notebook generates images from the trained textual inversion models generated with the `detsd.DetStableDiffusionTITrainer` class and saved as Determined checkpoints.  This notebook should be connected to a GPU.

### Pre-Launch Setup

A [Huggingface User Access Token](https://huggingface.co/docs/hub/security-tokens) is required to download the [Stable Diffusion weights](https://huggingface.co/CompVis/stable-diffusion-v1-4). To use this notebook, please modify the following lines in the `detsd-notebook.yaml` file:
```yaml
environment:
    environment_variables:
        - HF_AUTH_TOKEN=YOUR_HF_AUTH_TOKEN_HERE
```
after which this notebook can be launched by calling the below from the root of the repo directory
```bash
det -m MASTER_URL_WITH_PORT notebook start --config-file detsd-notebook.yaml --context .
```
and then opening up the copy of `textual_inversion.ipynb` on the master.

Update the jupyter notebook for better progress-bar rendering (may require a re-load to take effect). If the notebook was launched using the above command, other dependencies were already installed upon agent start-up through the repo's `startup-hook.sh` script.

In [ ]:
! pip install -qq jupyterlab-widgets==1.1.1 ipywidgets==7.7.2

## Creating the Pipeline

Import the `DetSDTextualInversionPipeline` class from `detsd.py` (loaded via the `--context` flag above), which will be used to generate Stable Diffusion images.

In [ ]:
from detsd import DetSDTextualInversionPipeline

Instantiate the pipeline with the default arguments:

In [ ]:
detsd_pipeline = DetSDTextualInversionPipeline()

Note: `DetSDTextualInversionPipeline` is initialized with `use_fp16=True` by default which increase inference speed and reduce memory usage, at the cost of somewhat reduced-quality images.  All available args can be viewed by uncommenting and running the cell below

In [ ]:
# ? DetSDTextualInversionPipeline

## Load Determined Checkpoints

We can now load textual-inversion checkpoints into the model. They are assumed to have been trained with `DetStableDiffusionTITrainer`, also contained in `detsd.py`.  These Determined checkpoints can be specified by their uuid, assuming all such checkpoints exist on the master we are currently logged into.

In [ ]:
# Code for logging into the master, if not already logged in.
# Not required if notebook was launched as described above.

# from determined.experimental import client
# client.login(master=MASTER_URL_WITH_PORT, user=USER, password=PASS)

Fill in the `uuids` list below with the `uuid` values of any Determined checkpoints you wish to incorporate into the model.

If no uuids are provided, images will be generated below using the vanilla Stable Diffusion model.

In [ ]:
uuids = []
detsd_pipeline.load_from_uuids(uuids)

## Generate Images

Finally, let's generate some art.

Create a directory in which to save the generated images:

In [ ]:
! mkdir generated_images

Grab the first concept which was loaded into the pipeline, if any, and store it as `first_concept`. If no checkpoints were loaded in, use `orange brain` as the default concept instead.

In [ ]:
all_added_concepts = detsd_pipeline.all_added_concepts
if all_added_concepts:
    first_concept = all_added_concepts[0]
else:
    first_concept = 'orange brain'
print(f'Using "{first_concept}" as first_concept in the below')
print(f'All available concepts: {all_added_concepts}')

The below code generates `num_samples = 4` images from the prompt in batches of size `batch_size=2`, returning the images as a list.

In [ ]:
# Generate and inspect images:
img_list = detsd_pipeline(prompt=f'a watercolor painting on textured paper of a {first_concept} using soft strokes, pastel colors, incredible composition, masterpiece',
                      num_samples=4,
                      batch_size=2,
                      num_inference_steps=50,
                      seed=2147483647,
                      guidance_scale=7.5,
                      saved_img_dir='generated_images')

In [ ]:
for img in img_list:
    display(img)

The three args below represent your primary knobs for altering the generated output:
* `num_inference_steps`: how many steps to run the generation process for. ~50 is typical
* `guidance_scale`: tunes how much weight is given to the prompt during generation. 7.5 is the default, with larger numbers leading to stronger adherence to the prompt.
* `generator_seed`: fixed RNG seed for reproducibility

Additional arguments can be passed to the underlying Huggingface `StableDiffusionPipeline` instance through the `other_hf_pipeline_call_kwargs` arg. See the [Hugging Face documentation](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion#diffusers.StableDiffusionPipeline.__call__) for information on all available arguments.

The three args below represent your primary knobs for altering the generated output:
* `num_inference_steps`: how many steps to run the generation process for. ~50 is typical
* `guidance_scale`: tunes how much weight is given to the prompt during generation. 7.5 is the default, with larger numbers leading to stronger adherence to the prompt.
* `generator_seed`: fixed RNG seed for reproducibility

Additional arguments can be passed to the underlying Huggingface `StableDiffusionPipeline` instance through the `other_hf_pipeline_call_kwargs` arg. See the [Hugging Face documentation](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion#diffusers.StableDiffusionPipeline.__call__) for information on all available arguments.